In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import re

ESTRS  = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'
CAUSAL = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Best_causality.Table'
CATALOG1='/storage/szfeupe/Runs/650GTEx_estr/gwas/gwas-catalog-associations.tsv'
CATALOG2='/storage/szfeupe/Runs/650GTEx_estr/gwas/NIHMS859943-supplement-3.csv'
STRGT = '/storage/szfeupe/Runs/650GTEx_estr/Genotypes/NormalizedGenotypes.table'
SNPGT = '/storage/szfeupe/Runs/650GTEx_estr/SNP_Analysis/'#chr13.tab
EXPRESSION='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'
header=["#EQTL_GENE","chrom","snp.start",'P-VALUE','OR_or_BETA','DISEASE/TRAIT']
DISTFROMSNP=100000
CHR = [str(i) for i in range(1,23,1)]+['X','Y']

In [7]:
def findstr(chrom,start):
    #strgt = strs.loc[strs['chrom']==chrom]
    cis_strs = strgt[(strgt["start"] >= (start-DISTFROMSNP)) & (strgt["start"] <= (start+DISTFROMSNP))]
    return(cis_strs)

LD01='/storage/szfeupe/Runs/650GTEx_estr/gwas/tmp/snp_str_gwas_ld_hg19_ALL.tab'

ld_data = pd.read_csv(LD01, sep='\s+').drop_duplicates().dropna()
print(ld_data.columns, ld_data.shape)
ld_data['pval'] = ld_data['pval'].astype(float)
ld_data.columns = ['str_locus', 'snp_locus','allele','freq_het','MAF','KL','r2','pval']
gwas_loci = ld_data.loc[(ld_data['pval'].astype(float)>0)&(ld_data['pval'].astype(float)<0.05)]
print(ld_data.shape, gwas_loci.shape)
gwas_loci

Index(['str_locus', 'snp_locus', 'allele', 'freq_het', 'MAF', 'KL', 'r2',
       'pval'],
      dtype='object') (3621, 8)
(3621, 8) (1106, 8)


,str_locus,snp_locus,allele,freq_het,MAF,KL,r2,pval
4,10:101188104,10:101188121,locus,0.127542,0.024540,None,0.014081,4.415554e-03
9,10:102674054,10:102674070,locus,0.499981,0.003834,None,0.880348,4.802327e-264
12,10:102915198,10:102915213,locus,0.645216,0.007669,None,0.007716,2.568558e-02
16,10:103111870,10:103111889,locus,0.446187,0.008436,None,0.006937,3.647324e-02
24,10:109567896,10:109567929,locus,0.643146,0.001057,None,0.009631,4.391430e-02
26,10:110263644,10:110263658,locus,0.354709,0.253834,None,0.054856,2.458606e-09
33,10:116015531,10:116015541,locus,0.521001,0.000767,None,0.010189,1.254876e-02
36,10:116754785,10:116754818,locus,0.549246,0.000767,None,0.006577,4.121784e-02
37,10:117473186,10:117473261,locus,0.820754,0.002301,None,0.007563,3.484410e-02
43,10:12005327,10:12005354,locus,0.174048,0.065184,None,0.824141,1.729504e-244


In [3]:
#CATALOG for SV
cat2 = pd.read_csv(CATALOG2)
#chrom, start, rsid, locusname, pval
cat2['chrom'] = cat2['GWAS_SNP_GTEX_ID'].apply(lambda x: x.split('_')[0],1)
cat2['snp.start'] = cat2['GWAS_SNP_GTEX_ID'].apply(lambda x: int(x.split('_')[1]),1)
cat2['rsid'] = cat2['GWAS_SNP_RSID'].astype(str)
cat2['locusname'] = cat2['DISEASE/TRAIT']
cat2['pval'] = cat2['P-VALUE'].astype(float)
cat2['snp_locus'] = cat2.apply(lambda x: x['chrom']+':'+str(int(x['snp.start'])) , 1)
#strs = pd.read_csv(STRGT, sep='\t')
#cat2[['chrom','snp.start','rsid','locusname','pval']].drop_duplicates().to_csv('/storage/szfeupe/Runs/650GTEx_estr/gwas/gwas_catalog_sv_loci.tab', sep='\t',index=None)

In [4]:
#GWAS LD OUTPUT   
trait_map = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/gwas/gwas_catalog_loci_all.tab', sep='\t')
trait_map['snp_locus'] = trait_map.apply(lambda x: x['chrom']+':'+str(x['snp.start']) , 1)
estrs = pd.read_csv(CAUSAL, sep='\t')
gwas_loci = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/gwas/str_snp_ld_01_up.tab', sep='\t')
gwas_loci = gwas_loci.loc[(gwas_loci['pval']>0)&(gwas_loci['pval']<0.050)].dropna()
gwas_loci

,str_locus,snp_locus,allele,freq_het,MAF,KL,r2,pval
2,1:7057322,1:7057337,locus,0.246394,0.158742,None,0.467730,7.061519e-89
5,1:14464578,1:14464603,locus,0.653780,0.486963,None,0.526591,1.086263e-98
7,1:15457306,1:15457349,locus,0.687972,0.013037,None,0.166286,9.219658e-26
9,1:16151270,1:16151281,locus,0.217375,0.105754,None,0.830187,4.544006e-215
10,1:17888911,1:17888926,locus,0.144272,0.075153,None,0.888368,2.720324e-309
13,1:21496684,1:21496710,locus,0.570353,0.052147,None,0.239932,1.255197e-39
18,1:34309878,1:34309889,locus,0.500563,0.457055,None,0.797763,7.160485e-222
20,1:36148600,1:36148629,locus,0.116725,0.009969,None,0.155691,8.698178e-23
22,1:40477739,1:40477749,locus,0.222672,0.106532,None,0.567516,5.346343e-102
24,1:46934172,1:46934202,locus,0.715619,0.029141,None,0.128536,5.321253e-20


In [7]:
estrs = estrs.dropna()
estrs['str_locus'] = estrs.apply(lambda x: x['chrom'][3:]+':'+str(int(x['best.str.start'])) , 1)
estrs = estrs.sort_values('str_locus')
print(estrs.shape, gwas_loci.shape)
estrs['best.score'] = estrs['best.score'].apply(lambda x: 100*x , 1)
Subset = estrs.merge(gwas_loci, on=['str_locus'], how='inner')
print(Subset.shape)
Subset.to_csv('/storage/szfeupe/Runs/650GTEx_estr/gwas/GWAS_OUT1.tab', sep='\t', index=None)

(11810, 11) (384, 8)
(27, 18)


In [8]:
cat2['snp_locus'] = cat2.apply(lambda x: x['chrom']+':'+str(int(x['snp.start'])) , 1)
cat=cat2[['chrom','snp.start','rsid','locusname','pval', 'snp_locus']].copy()
svset = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/gwas/str_sv_tudy_gwas_ld.tab', sep='\t').drop_duplicates()
svset.columns = ['str_locus', 'snp_locus','allele','freq_het','MAF','KL','r2','pval']
GWAS2 = svset.merge(estrs, on =['str_locus'], how='inner')
print(svset.shape, GWAS2.shape)
GWAS2

(483, 8) (122, 18)


In [4]:
#gwas loci     chrom, start, rsid, locusname, pval
print('get gwas ...')
gwas1 = pd.read_csv(CATALOG1, sep='\t',low_memory=False )
print(gwas1.shape)
gwas = gwas1[['CHR_ID', 'CHR_POS', 'SNPS','DISEASE/TRAIT','P-VALUE']]
gwas['CHR_ID']=gwas['CHR_ID'].astype(str)
gwas = gwas.dropna()

CHROM = sorted(set(gwas['CHR_ID'].astype(str)))
#gwas['CHR_ID'] = gwas['CHR_ID'].apply(lambda x: x.split(';')[0])
df = gwas.loc[gwas['CHR_ID'].isin(CHR)].copy()
print(df.shape)
df2 = gwas.loc[~gwas['CHR_ID'].isin(CHR)].copy()
print(df2.shape)

end=-1
for index, row in df2.iterrows():
    if len(row['CHR_ID'].split('x'))<=1 : continue
    for i in range(2):
        #print(i,'...',[row['CHR_ID'].split('x')[i].strip(),row['CHR_POS'].split('x')[i].strip(), row['SNPS'].split('x')[i].strip()])
        df.loc[end] = [row['CHR_ID'].split('x')[i].strip().strip(),row['CHR_POS'].split('x')[i].strip(), row['SNPS'].split('x')[i].strip(),row['DISEASE/TRAIT'],'-1'] 
        end = end-1        
print(' x ',df.shape)        
        
end = end-1               
for index, row in df2.iterrows():
    if len(row['CHR_ID'].split(';'))<2: continue
    for i in range(len(row['CHR_ID'].split(';'))):
        #print(i,'...',[row['CHR_ID'].split(';')[i].strip(),row['CHR_POS'].split(';')[i].strip(), row['SNPS'].split(';')[i].strip()])
        df.loc[end] = [row['CHR_ID'].split(';')[i].strip().strip(),row['CHR_POS'].split(';')[i].strip(), row['SNPS'].split(';')[i].strip(),row['DISEASE/TRAIT'],'-1'] 
        end = end-1                       
print(' ; ',df.shape)        

df.columns = ['chrom', 'snp.start', 'rsid','locusname', 'pval']
df['rsid']=df['rsid'].astype(str)
#df['chrom'] = df['chrom'].apply(lambda x: 'chr%s'%str(x), 1)
print(list(set(df['chrom'])) )
df.to_csv('/storage/szfeupe/Runs/650GTEx_estr/gwas/gwas_catalog_loci_all.tab', index=None, sep='\t')

get gwas ...
(34094, 34)
(33063, 5)
(261, 5)


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


 x  (33353, 5)
 ;  (33809, 5)
['20', '13', '7', '22', 'X', '15', '8', '12', '19', '5', '1', '16', '9', '10', '4', '11', '3', '6', '18', '21', '2', '14', '17']
